# Transition states
In this notebook we will locate a transition state by using interpolated force fields from reactant to product(s).

In [ ]:
import veloxchem as vlx
import time

In [ ]:
rea = vlx.Molecule.read_smiles('C1=CCCCC1')
pro1 = vlx.Molecule.read_smiles('C=CC=C')
pro2 = vlx.Molecule.read_smiles('C=C')
rea.show()
pro1.show()
pro2.show()

In [ ]:
start_time = time.time()
EVB = vlx.EvbDriver()
# EVB.ostream.mute()
EVB.build_ff_from_molecules(rea, [pro1, pro2])
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "BLYP" 
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
ts_guesser = vlx.TransitionStateGuesser()
ts_guesser.scf_drv = scf_drv
#some options to tune the algorithm, don't have to be bothered with if not necessary
# ts_guesser.lambda_vec = np.round(np.linspace(0, 1, 21), 3)
ts_guesser.scf_basis = 'def2-sv(p)'  # Default is def2-SVP
# ts_guesser.mute_scf = True
# ts_guesser.mm_temperature = 600
#ts_guesser.mm_steps = 50
# ts_guesser.mm_step_size = 0.001 * mmunit.picoseconds

TS_mol, results = ts_guesser.find_TS(
    EVB,
    scf=True,
)

ts_guesser.show_results()


In [ ]:
basis = vlx.MolecularBasis.read(TS_mol, 'def2-sv(p)')

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = 'blyp'
scf_drv.ri_coulomb = True
scf_results = scf_drv.compute(TS_mol, basis)

opt_drv = vlx.OptimizationDriver(scf_drv)
opt_drv.conv_energy = 1e-04
opt_drv.conv_drms = 1e-02
opt_drv.conv_dmax = 2e-02
opt_drv.conv_grms = 4e-03
opt_drv.conv_gmax = 8e-03
opt_drv.transition = True
opt_results = opt_drv.compute(TS_mol, basis, scf_results)

In [ ]:
print(time.time() - start_time)

In [ ]:
opt_drv.show_convergence(opt_results)